# Lesson 1: Router Engine

## Setup

In [1]:
from custom_utils.helper import get_openai_api_key, print_response

OPENAI_API_KEY = get_openai_api_key()

In [2]:
import nest_asyncio

nest_asyncio.apply()

## Load Data

In [3]:
from llama_index.core import SimpleDirectoryReader

# load documents
documents = SimpleDirectoryReader(input_files=["../resources/metagpt.pdf"]).load_data()

## Define LLM and Embedding model

In [4]:
from llama_index.core.node_parser import SentenceSplitter

splitter = SentenceSplitter(chunk_size=1024)
nodes = splitter.get_nodes_from_documents(documents)

In [5]:
from llama_index.core import Settings
from llama_index.llms.openai import OpenAI
from llama_index.embeddings.openai import OpenAIEmbedding

Settings.llm = OpenAI(model="gpt-3.5-turbo")
Settings.embed_model = OpenAIEmbedding(model="text-embedding-ada-002")

## Define Summary Index and Vector Index over the Same Data

In [6]:
from llama_index.core import SummaryIndex, VectorStoreIndex

summary_index = SummaryIndex(nodes)
vector_index = VectorStoreIndex(nodes)

## Define Query Engines and Set Metadata

In [7]:
summary_query_engine = summary_index.as_query_engine(
    response_mode="tree_summarize",
    use_async=True,
)
vector_query_engine = vector_index.as_query_engine()

In [8]:
from llama_index.core.tools import QueryEngineTool


summary_tool = QueryEngineTool.from_defaults(
    query_engine=summary_query_engine,
    description=(
        "Useful for summarization questions related to MetaGPT"
    ),
)

vector_tool = QueryEngineTool.from_defaults(
    query_engine=vector_query_engine,
    description=(
        "Useful for retrieving specific context from the MetaGPT paper."
    ),
)

## Define Router Query Engine

In [9]:
from llama_index.core.query_engine.router_query_engine import RouterQueryEngine
from llama_index.core.selectors import LLMSingleSelector


query_engine = RouterQueryEngine(
    selector=LLMSingleSelector.from_defaults(),
    query_engine_tools=[
        summary_tool,
        vector_tool,
    ],
    verbose=True
)

In [10]:
response = query_engine.query("What is the summary of the document?")
print_response(str(response))

Selecting query engine 0: This choice indicates that the document is useful for summarization questions related to MetaGPT..


The document introduces MetaGPT, a meta-programming framework that utilizes Standardized Operating Procedures (SOPs) to enhance multi-agent systems based on Large Language Models (LLMs). MetaGPT focuses on role specialization, workflow management, and efficient communication mechanisms to improve code generation quality. Through experiments, MetaGPT demonstrates top-tier performance on various benchmarks in software development tasks. It also discusses potential future enhancements like self-improvement mechanisms and multi-agent economies. The document details the development process of a "Drawing App" using MetaGPT, highlighting technical specifications, system design, and implementation using Python libraries. It outlines the roles of different agents in the development cycle and compares MetaGPT's code generation performance with other models. Additionally, it addresses the impact of instruction levels on task outcomes, challenges overcome by MetaGPT's designs, ethical concerns, limitations, and the potential influence of MetaGPT on programming accessibility and transparency.

In [11]:
print(len(response.source_nodes))

34


In [12]:
response = query_engine.query(
    "How do agents share information with other agents?"
)
print_response(str(response))

Selecting query engine 1: This choice is more relevant as it specifically mentions retrieving specific context, which is necessary for understanding how agents share information with other agents..


Agents share information with other agents by utilizing a shared message pool where they can publish structured messages. This shared message pool allows all agents to exchange messages directly, enabling them to both publish their own messages and access messages from other entities transparently. Additionally, agents can subscribe to relevant messages based on their role profiles, allowing them to extract the information that is most pertinent to their specific roles.

## Let's put everything together

In [13]:
from custom_utils.utils import get_router_query_engine

query_engine = get_router_query_engine("../resources/metagpt.pdf")

In [14]:
response = query_engine.query("Tell me about the ablation study results?")
print_response(str(response))

Selecting query engine 1: Ablation study results are specific context from the MetaGPT paper, making choice 2 the most relevant..


The ablation study results demonstrate the effectiveness of MetaGPT in addressing challenges related to context utilization, reducing hallucinations in software generation, and managing information overload. The study highlights how MetaGPT's unique designs successfully tackle issues such as ambiguity in natural language descriptions, maintaining information validity in lengthy contexts, reducing code hallucinations, and handling information overload through a global message pool and subscription mechanism.